In [ ]:
from keras.models import Model
from keras.layers import Input
from ae_model import encoder_vgg16, decoder1_vgg16, decoder2_vgg16, decoder3_vgg16, encoder1_vgg16

input_shape = Input(shape=[256, 192, 3])
ae = Model(input_shape, decoder1_vgg16(encoder_vgg16(input_shape)))
ae.summary()

In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from keras.preprocessing.image import img_to_array, load_img

datasets = []
path = './grid-search/dataset-bc/JPEGImages/'

files = os.listdir(path)


for i in tqdm(files):
    img = load_img(path+i, target_size=(192, 256))
    datasets.append(img_to_array(img))

data_array = np.reshape(datasets, (len(datasets), 256, 192, 3))
data_array = data_array.astype('float32') / 255.

print(img.size)
print(data_array.shape)

100%|██████████| 366/366 [00:00<00:00, 752.23it/s]


(256, 192)
(366, 256, 192, 3)


In [ ]:
from keras.callbacks import TensorBoard
import tensorflow as tf
from datetime import datetime
%load_ext tensorboard

batch_size = [8, 16 ,32]
epoch = [50, 100]

for bb in batch_size:
    for ee in epoch:

        log_dir = './grid-search/logs/find_batchs_and_epochs/' + str(bb) + '_' + str(ee) + 'epochs'
        print('logs save in:', log_dir)
        work_text = 'Adam(Default Config)_MSE(Default Config)_Accuracy(mereics)____@ ' + str(bb) + 'batch_sizes___' + str(ee) + 'epochs!!!____@' + datetime.now().strftime("%Y%m%d-%H%M%S")
        fw =  tf.summary.create_file_writer(log_dir)
        with fw.as_default():
            tf.summary.text("config_record", work_text, step=0)
        ae.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
        ae.fit(data_array, data_array, epochs=ee, batch_size=bb, shuffle=True, verbose=1, callbacks=[TensorBoard(log_dir=log_dir)])

